In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
import tqdm
import torch.nn.functional as F
import tqdm
import impute
import PMLE
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import datetime
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools.tools import add_constant

/Users/remycanario/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)
Using TensorFlow backend.


# Data Preprocessing

In [5]:
df = pd.read_csv('testing_df.csv',index_col=0)
df.drop('TransactionID',axis=1,inplace=True)


/Users/remycanario/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (24,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1. Prune Columns

In [6]:
dropped_cols = []

#drop columns that have excessive NA 
for col in df.columns:
    if df[col].isnull().sum()>0.95*df.shape[0]:
        df.drop(col,axis=1,inplace=True)
        dropped_cols.append(col)
        
#drop columns that have only one value
for col in df.min()[df.min()==df.max()].index:
    df.drop(col,axis=1,inplace=True)
    dropped_cols.append(col)

## 2. Feature Engineering

In [7]:
X = df.drop('isFraud',axis=1)
y = df['isFraud']

#Create feature for number of NAs per column
row_nas = []
for i in range(X.shape[0]):
    row_nas.append(X.iloc[i,:].isna().sum())
X['NAs']= row_nas

#Create features of characteristics of transaction amounts
X['amtby5']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[0]) % 5==0 else 0)
X['centsby5']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1]) % 5==0 else 0)
X['threedec']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1]) > 99 else 0)
X['centsnot0']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1][:2])!=0 else 0)
X['centsnot95']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1][:2])!=95 else 0)
X['digits']= X.TransactionAmt.apply(lambda x: str(x).split('.')[0][-1])
X['places']= X.TransactionAmt.apply(lambda x: str(len(str(x).split('.')[0])))


#Prepare card columns for one hot encoding 
X.loc[:,'card1':'card5'].astype(str)

#Convert TransactionID into datetime and derive features
X['Datetime'] = X.TransactionDT.apply(lambda x:datetime.datetime(2018,1,1)+datetime.timedelta(seconds=x))
X['Weekday']= X.Datetime.apply(lambda x: x.weekday())
X['Month']= X.Datetime.apply(lambda x: x.month)
X['Hour']= X.Datetime.apply(lambda x: x.hour)
X['Day']=(X.TransactionDT - 86400)//(60*60*24)
X['Week']= X.Day//7
X['Minute']= X.Datetime.apply(lambda x: x.minute)
X.drop('Datetime',axis=1,inplace=True)

X = pd.get_dummies(X,drop_first=True,dummy_na=True)

## 3. Train Test Split

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=.2,random_state=0)

## 4. Fill Missing Values with Multiple Imputation

In [9]:
#Establish floors and ceilings for column values
column_ranges = {}
for col in X_train.columns:
    column_ranges[col]=(X_train[col].min(),X_train[col].max())

#Impute missing values using multiple imputation
X_train, train_problems = impute.impute_missing_values(X_train,col_floor_ceiling_dict=column_ranges)
X_val, val_problems = impute.impute_missing_values(X_val,col_floor_ceiling_dict=column_ranges)

print('X_train NAs = ', X_train.isna().sum().sum())
print('X_val NAs = ', X_train.isna().sum().sum())

  0%|          | 0/314 [00:00<?, ?it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  0%|          | 1/314 [00:01<05:23,  1.03s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  1%|          | 2/314 [00:02<05:37,  1.08s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

  7%|▋         | 22/314 [00:22<04:33,  1.07it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  7%|▋         | 23/314 [00:22<04:26,  1.09it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  8%|▊         | 24/314 [00:23<04:26,  1.09it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

 14%|█▍        | 44/314 [00:44<04:54,  1.09s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 14%|█▍        | 45/314 [00:45<04:42,  1.05s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 15%|█▍        | 46/314 [00:47<05:14,  1.17s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

 21%|██        | 66/314 [01:10<05:05,  1.23s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 21%|██▏       | 67/314 [01:12<05:08,  1.25s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 22%|██▏       | 68/314 [01:13<05:00,  1.22s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

 28%|██▊       | 88/314 [01:37<04:33,  1.21s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 28%|██▊       | 89/314 [01:38<04:23,  1.17s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 29%|██▊       | 90/314 [01:39<04:28,  1.20s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

...

 75%|███████▌  | 236/314 [03:05<02:06,  1.62s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 75%|███████▌  | 237/314 [03:06<01:51,  1.45s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 76%|███████▌  | 238/314 [03:07<01:50,  1.46s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

 82%|████████▏ | 258/314 [03:36<01:32,  1.66s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 82%|████████▏ | 259/314 [03:37<01:22,  1.50s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 83%|████████▎ | 260/314 [03:38<01:22,  1.52s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

 89%|████████▉ | 280/314 [04:10<00:51,  1.51s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 89%|████████▉ | 281/314 [04:12<00:53,  1.61s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 90%|████████▉ | 282/314 [04:13<00:46,  1.46s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

 96%|█████████▌| 302/314 [04:47<00:23,  1.92s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 96%|█████████▋| 303/314 [04:49<00:20,  1.83s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 97%|█████████▋| 304/314 [04:50<00:16,  1.62s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

X_train NAs =  0
X_val NAs =  0


In [14]:
X_train = pd.read_csv('X_train_imputed.csv',index_col=0)
X_val = pd.read_csv('X_val_imputed.csv',index_col=0)
y = pd.read_csv('testing_df.csv',index_col=0).isFraud

y_train, y_val = train_test_split(y,test_size=.2,random_state=0)

/Users/remycanario/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (24,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 5. Standardize

In [15]:
#Save meta data
columns = X_train.columns
normal_inds = y_train[y_train==0].reset_index().index

#Standardize dfs
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_val_sc = sc.fit_transform(X_val)

# Autoencoder Anomaly Detection
### With undercomplete, regularized and denoising autoencoders using pytorch

## Data Processing

In [16]:
n_features = X_train_sc.shape[1]

#Convert non-fraud rows to pytorch tensor
normal_train = X_train_sc[normal_inds,:]
normal_torch = torch.from_numpy(normal_train,).type(torch.FloatTensor)

#Convert train and val sets to pytorch tensor
train_torch = torch.from_numpy(X_train_sc).type(torch.FloatTensor)
val_torch = torch.from_numpy(X_val_sc).type(torch.FloatTensor)

In [17]:
import autograd

In [7]:
!pip install autograd

  Created wheel for autograd: filename=autograd-1.3-cp37-none-any.whl size=47990 sha256=5c1a2f41945d8d032b6d2d80acd7b5bcd865fe8bcd18553878384f3f1e5d07dd
  Stored in directory: /Users/remycanario/Library/Caches/pip/wheels/42/62/66/1121afe23ff96af4e452e0d15e68761e3f605952ee075ca99f
Successfully built autograd


## Create Pytorch objects

In [18]:
class AutoEncoder(nn.Module):
    
    def __init__(self,n_features,hidden_nodes,dropout=None,VAE=False):
        
        '''PARAMATERS
           n_features: number of X variables in dataset
           hidden_nodes: number of nodes in hidden layer
           dropout: fraction of nodes to dropout (0 < dropout <1)'''
        
        super(AutoEncoder, self).__init__()
        self.n_features=n_features
        self.n_hidden = hidden_nodes
        self.encoder = nn.Linear(n_features,hidden_nodes)
        self.decoder = nn.Linear(hidden_nodes,n_features)
        self.output_layer = nn.Linear(n_features,n_features)
        self.dropout = dropout
        self.best_recon = None
        
        
    def forward(self,x):
        if self.dropout!=None:
            x = F.relu(F.dropout(self.encoder(x)))
        else:
            x = F.relu(self.encoder(x))
        self.hidden_layer=x
        x = F.relu(self.decoder(x))
        x = self.output_layer(x)
        return x
    
    def L1_loss(self,recon,inputs,lmbda):
        MSELoss = nn.MSELoss()
        loss = MSELoss(recon,inputs)
        for param in ae5.parameters():
            loss += lmbda*torch.sum(torch.abs(param))
        return loss


In [98]:
def train_autoencoder(model, dataset, loss_func, optimizer, epochs=100, batch_size=1024,  
                      validation_tensor=None,y_val=None, noise_factor=None, 
                      random_seed=None, MSE_stopping_threshold=0):
    '''Parameters
       model: instantiated autoencoder
       dataset: torch tensor of X variables
       loss_func: instantiated loss function
       optimizer: instantiated optimizer
       validation_tensor: torch tensor of validation X variables
       y_val: numpy array of y values
       epochs: number of epochs
       batch_size: batch_size
       noise_factor: magnitude of noise added to data
         for a denoising autoencoder (0 < noise_factor <=1)
       random_seed: random_seed
       stopping_MSE_threshold: MSE value after which autoencoder stops training'''
    
    if random_seed!=None:
        torch.manual_seed(random_seed)
    train_loader = torch.utils.data.DataLoader(dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True)
    
    if type(validation_tensor)==torch.Tensor:
        val = True
        val_numpy = validation_tensor.detach().numpy()
    else:
        val = False
        
    readout_batch_interval = 0.25*(dataset.shape[0]/batch_size)//1
    
    for epoch in range(epochs):
        counter = 0
        print('\n\033[1mEpoch {}\033[0m\n'.format(epoch+1))
        for batch in train_loader:
            
            if noise_factor!=None:
                batch = batch + noise_factor * torch.randn(*batch.shape)
            batch = torch.autograd.Variable(batch)
            optimizer.zero_grad()
            recon = model(batch)
            loss = loss_func(recon, batch)
            
            if (counter%readout_batch_interval==0):
                print('Batch {} Loss: {:.4f}'.format(counter, float(loss)))
            loss.backward()
            optimizer.step()
            
            counter+=1
        if epoch==0:
            epoch_loss = loss_func(model(dataset), dataset)
            print('\nEPOCH {} LOSS: {:.4f}'.format(epoch+1, float(epoch_loss)))
        else:
            old_epoch_loss = epoch_loss
            epoch_loss = loss_func(model(dataset), dataset)
            print('\nEPOCH {} LOSS: {:.4f}'.format(epoch+1, float(epoch_loss)))
            print('\nChange: {:.4f}%'.format(float((epoch_loss-old_epoch_loss)/old_epoch_loss)))
        if val==True:
            val_output = model(validation_tensor).detach().numpy()
            reconstruction_error = np.sqrt(np.power(val_output - val_numpy, 2)).sum(axis=1)
            reconstruction_error = reconstruction_error/(reconstruction_error.max()-reconstruction_error.min())
            if epoch==0:
                logit = Logit(y_val, add_constant(reconstruction_error)).fit(disp=0)
                score = logit.prsquared
                model.best_recon=model.parameters()
            else:
                old_score = score
                logit = Logit(y_val, add_constant(reconstruction_error)).fit(disp=0)
                score = logit.prsquared
                score =np.corrcoef(y_val,reconstruction_error)[0][1]
                if score>old_score:
                    model.best_recon=model.parameters()
                
            print('\nReconstruction error McFadden Pseudo-R2: {:.4f}'.format(score))
        if epoch_loss<=MSE_stopping_threshold:
            break

## Train Autoencoders

### Autoencoder I: Dropout

#### Instantiate encoder and loss function

In [99]:
ae1 = AutoEncoder(n_features,int(n_features*1.5//1),dropout=0.3)
loss_func = nn.MSELoss()

#### Train autoencoder

In [100]:
#Start with high learning rate
optimizer = torch.optim.SGD(ae1.parameters(), lr=0.2, momentum=0.9,nesterov=True)
train_autoencoder(model=ae1,
                   dataset=normal_torch,
                   loss_func=loss_func,
                   optimizer=optimizer,
                   batch_size=256,
                   epochs=150,
                   validation_tensor=val_torch,
                   y_val = y_val,
                   MSE_stopping_threshold=0.19)


Epoch 1

Batch 0 Loss: 0.9037
Batch 10 Loss: 0.8863
Batch 20 Loss: 0.8439
Batch 30 Loss: 0.9498
Batch 40 Loss: 0.7669

EPOCH 1 LOSS: 0.8979

Reconstruction error McFadden Pseudo-R2: 0.0289

Epoch 2

Batch 0 Loss: 0.8769
Batch 10 Loss: 0.8829
Batch 20 Loss: 0.9267
Batch 30 Loss: 0.9506
Batch 40 Loss: 0.9038

EPOCH 2 LOSS: 0.8683

Change: -0.0330%

Reconstruction error McFadden Pseudo-R2: 0.1668

Epoch 3

Batch 0 Loss: 0.8773
Batch 10 Loss: 0.7961
Batch 20 Loss: 0.9668
Batch 30 Loss: 0.7110
Batch 40 Loss: 0.8716

EPOCH 3 LOSS: 0.8334

Change: -0.0402%

Reconstruction error McFadden Pseudo-R2: 0.1994

Epoch 4

Batch 0 Loss: 0.7763
Batch 10 Loss: 0.7648
Batch 20 Loss: 0.7869
Batch 30 Loss: 0.7148
Batch 40 Loss: 0.8142

EPOCH 4 LOSS: 0.8013

Change: -0.0385%

Reconstruction error McFadden Pseudo-R2: 0.1860

Epoch 5

Batch 0 Loss: 0.6561
Batch 10 Loss: 0.8506
Batch 20 Loss: 0.8280
Batch 30 Loss: 0.8212
Batch 40 Loss: 0.7603

EPOCH 5 LOSS: 0.7784

Change: -0.0285%

Reconstruction error McFad

KeyboardInterrupt: 

#### Save results

In [115]:
ae1.parameters=ae1.best_recon
output1 = ae1(normal_torch)
train_output1 = ae1(train_torch)
val_output1 = ae1(val_torch)

### Autoencoder II: Undercomplete

In [116]:
ae2 = AutoEncoder(n_features,int(n_features*0.6)//1)
loss_func = nn.MSELoss()

In [117]:
optimizer = torch.optim.SGD(ae2.parameters(), lr=0.2,momentum=0.9,nesterov=True)
train_autoencoder(model=ae2,
                   dataset=output1,
                   loss_func=loss_func,
                   optimizer=optimizer,
                   batch_size=1024,
                   epochs=120,
                   validation_tensor=val_torch,
                   y_val=y_val,
                   MSE_stopping_threshold=0.05)


Epoch 1

Batch 0 Loss: 0.2615
Batch 2 Loss: 0.2164
Batch 4 Loss: 0.2980
Batch 6 Loss: 0.2905
Batch 8 Loss: 0.2142
Batch 10 Loss: 0.2049

EPOCH 1 LOSS: 0.2521

Reconstruction error McFadden Pseudo-R2: 0.0150

Epoch 2

Batch 0 Loss: 0.2593
Batch 2 Loss: 0.2343
Batch 4 Loss: 0.2604
Batch 6 Loss: 0.2108
Batch 8 Loss: 0.2885
Batch 10 Loss: 0.2950

EPOCH 2 LOSS: 0.2300

Change: -0.0876%

Reconstruction error McFadden Pseudo-R2: 0.1036

Epoch 3

Batch 0 Loss: 0.2286
Batch 2 Loss: 0.2268
Batch 4 Loss: 0.2354
Batch 6 Loss: 0.2385
Batch 8 Loss: 0.1922
Batch 10 Loss: 0.2872

EPOCH 3 LOSS: 0.2069

Change: -0.1003%

Reconstruction error McFadden Pseudo-R2: 0.1350

Epoch 4

Batch 0 Loss: 0.2074
Batch 2 Loss: 0.1911
Batch 4 Loss: 0.2036
Batch 6 Loss: 0.1918
Batch 8 Loss: 0.2131
Batch 10 Loss: 0.1842

EPOCH 4 LOSS: 0.1978

Change: -0.0440%

Reconstruction error McFadden Pseudo-R2: 0.1631

Epoch 5

Batch 0 Loss: 0.1645
Batch 2 Loss: 0.1827
Batch 4 Loss: 0.2433
Batch 6 Loss: 0.1951
Batch 8 Loss: 0.1841

KeyboardInterrupt: 

#### Save results

In [118]:
ae2.parameters=ae2.best_recon
output2 = ae2(output1)
train_output2 = ae2(train_output1)
val_output2 = ae2(val_output1)

### Autoencoder III: Denoising

In [124]:
ae3 = AutoEncoder(n_features,n_features)
loss_func = nn.MSELoss()

In [125]:
optimizer = torch.optim.SGD(ae3.parameters(), lr=0.2,momentum=0.9,nesterov=True)
train_autoencoder(model=ae3,
                  dataset=output2,
                  loss_func=loss_func,
                  optimizer=optimizer,
                  batch_size=512,
                  epochs=100,
                  noise_factor=0.75,
                  validation_tensor=val_torch,
                  y_val=y_val)


Epoch 1

Batch 0 Loss: 0.6658
Batch 5 Loss: 0.7112
Batch 10 Loss: 0.6895
Batch 15 Loss: 0.7023
Batch 20 Loss: 0.6814

EPOCH 1 LOSS: 0.1244

Reconstruction error McFadden Pseudo-R2: 0.0141

Epoch 2

Batch 0 Loss: 0.6996
Batch 5 Loss: 0.6507
Batch 10 Loss: 0.6474
Batch 15 Loss: 0.6455
Batch 20 Loss: 0.6807

EPOCH 2 LOSS: 0.0902

Change: -0.2750%

Reconstruction error McFadden Pseudo-R2: 0.1683

Epoch 3

Batch 0 Loss: 0.6298
Batch 5 Loss: 0.7323
Batch 10 Loss: 0.6344
Batch 15 Loss: 0.6341
Batch 20 Loss: 0.6726

EPOCH 3 LOSS: 0.0746

Change: -0.1733%

Reconstruction error McFadden Pseudo-R2: 0.1838

Epoch 4

Batch 0 Loss: 0.6253
Batch 5 Loss: 0.6192
Batch 10 Loss: 0.6576
Batch 15 Loss: 0.6434
Batch 20 Loss: 0.6108

EPOCH 4 LOSS: 0.0554

Change: -0.2566%

Reconstruction error McFadden Pseudo-R2: 0.2074

Epoch 5

Batch 0 Loss: 0.6006
Batch 5 Loss: 0.6026
Batch 10 Loss: 0.6014
Batch 15 Loss: 0.6126
Batch 20 Loss: 0.5953

EPOCH 5 LOSS: 0.0419

Change: -0.2442%

Reconstruction error McFadden P

KeyboardInterrupt: 

In [126]:
#get outputs
ae3.parameters=ae2.best_recon
output3 = ae3(output2)
train_output3 = ae3(train_output2)
val_output3 = ae3(val_output2)

### Autoencoder IV: L2 Regularization

In [ ]:
ae4 = AutoEncoder(n_features,n_features)

loss_func = L1_loss

In [129]:
lmbda=0.25
optimizer = torch.optim.SGD(ae4.parameters(), lr=1e-8,momentum=0.9,nesterov=True)
train_autoencoder(model=ae4,
                   dataset=output4,
                   loss_func=loss_func,
                   optimizer=optimizer,
                   batch_size=1024,
                   epochs=120,
                   validation_tensor=val_torch,
                   MSE_stopping_threshold=0.05)

In [261]:
#get outputs
output4 = ae4(output3)
train_output4 = ae4(train_output3)
val_output4 = ae4(val_output3)

## Autoencoder V: L2 Regularization

In [133]:
ae5 = AutoEncoder(n_features,n_features)
loss_func = nn.MSELoss()

In [ ]:
optimizer = torch.optim.RMSprop(ae5.parameters(), lr=0.02,weight_decay=0.1)
train_autoencoder(model=ae5,
                  dataset=output4,
                  loss_func=loss_func,
                  optimizer=optimizer,
                  batch_size=1024,
                  epochs=1000,
                  validation_tensor=val_torch,
                  y_val=y_val)

In [326]:
output5 = ae5(output4)
train_output5 = ae5(train_output4)
val_output5 = ae5(val_output4)

In [ ]:
final_train_output = train_output5.detach().numpy()
final_val_output = train_output5.detach().numpy()
train_reconstruction_score = sc.fit_transform(np.power(X_train_sc - final_train_output,2).sum(axis=1))
val_reconstruction_score = sc.fit_transform(np.power(X_val_sc - final_val_output,2).sum(axis=1))

# Penalized Maximum Likelihood Estimation
Using Firth's Logistic Regression with Intercept Correction

In [ ]:
FLIC = PMLE.PMLE.Firth_Logit(FLIC=True)
FLIC.fit(X_train_sc,y_train)
FLIC_train_preds = FLIC.predict_proba(X_train_sc)
FLIC_val_preds = FLIC.predict_proba(X_train_sc)

# SMOTE Dataset Resampling
Estimation with XGBoost

## Perform Synthetic Minority Over-Sampling Technique

In [ ]:
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train_sc,y_train)
X_val_smote, y_val_smote = smote.fit_resample(X_val_sc,y_val)

## Get XGBoost estimate of SMOTE dataset

In [ ]:
#Instantiate XGBoost and grid search
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
xgb_gs = GridSearchCV(xgb,params,cv=3)

#fit model and obtain predictions
xgb_gs.fit(X_train_smote,y_train_smote)

smote_train_preds = xgb_gs.predict_proba(X_train_smote)
smote_val_preds = xgb_gs.predict_proba(X_val_smote)

# Ensemble results

## Create Ensemble DFs

In [ ]:
ensemble_X_train = pd.DataFrame(None)
ensemble_X_train['anomaly_score'] = train_reconstruction_score
ensemble_X_train['FLIC'] = FLIC_train_preds
ensemble_X_train['smote'] = smote_train_preds

ensemble_X_val = pd.DataFrame(None)
ensemble_X_val['anomaly_score'] = val_reconstruction_score
ensemble_X_val['FLIC'] = FLIC_val_preds
ensemble_X_val['smote'] = smote_val_preds

## Get results using FLIC 

In [ ]:
FLIC.fit(ensemble_X_train,y_train)
preds = FLIC.predict_proba(ensemble_X_val)